# Chapter3. A Tour of Machine Learning <u>Classifiers</u> Using Scikit-learn

Chapter2에서는 Perceptrons와 Adaptive linear neurons을 이용해 간단한 <u>Classification problem</u>을 다뤄보았다. 

Chapter3에서는 <u>Classification problem</u>을 풀 수 있는     
보다 현대적인 방법론들(logistic regression, SVM, K-nearest neighbors, ...)에 대해 간단하게 다뤄보고     
이런 방법들을 Python의 Machine Learning library인 Scikit-learn을 이용해 구현해볼 것이다.

### Choosing a <u>classification algorithm</u>

그것이 Regression problem이든 Classification problem이든 하나의 문제를 푸는 다양한 방법론이 존재한다. 앞서 Chapter2에서 같은 문제를 Perceptrons 혹은 Adaptive linear neurons를 이용해 풀 수 있었다. 이밖에도 다양한 방법론이 존재한다.

다양한 방법론이 있다면 그냥 가장 좋은 것을 사용하면 되지 않은가? 라고 생각할 수도 있다.

하지만 각 방법론마다 데이터에 대한 다른 조건을 가정하고 있고,     
문제를 풀어가는 방식이 조금식 다르기 때문에     
어떤 데이터에는 좋은 방법이 다른 데이터에는 좋지 않을수도 있는 것이다.   
(여기서 다른 데이터라는 것은 변수(feature)의 수가 다르거나 sample수가 다르거나 noise가 다르거나, 변수간의 관계가 다르거나 다양한 상황이 있을 있다.)

따라서 데이터에 따라 좋은 방법을 잘 골라 쓰는 것이 중요하고,      
해당 데이터에 잘 들어 맞는 방법을 고르는 것도 하나의 테크닉이라고 할 수 있다.


### First steps with scikit-learn

다양한 방법론을 소개하면서 이들을 scikit-learn을 이용해 구현해 볼텐데,    

우선 Iris flower 데이터를 Chapter2에서 다뤘던 Perceptrons와 Adaline을 scikit-learn으로 구현해보자.

In [1]:
from sklearn import datasets
import numpy as np
iris = datasets.load_iris()
X = iris.data[:, [2, 3]]
y = iris.target


> 먼저 feature matrix $\mathbf{X}$와 target vector $\mathbf{y}$에 로드 한다.

In [2]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

> 데이터 $[\mathbf{X}:\mathbf{y}]$를 (random하게) training data와 test data로 7:3의 비율로 random하게 나눈다.      

> training data$[\mathbf{X}_{train}:\mathbf{y}_{train}]$는 모형을 학습시키는데 사용되고,         
test data$[\mathbf{X}_{test}:\mathbf{y}_{test}]$는 모형의 성능을 평가하는데 사용된다.          

In [3]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

> 최적의 성능을 위해서는 각 feature들을 **feature scaling**을 하며 다양한 기법 중 **standardization**(표준화)의 경우 아래와 같이 각 변수에서 평균($\mu$)을 빼고 표준편차($\sigma$)로 나눠 정규분포 $N(0, 1)$로 변환한다.
$$x^{'}_j = \frac{x_j - \mu_j}{\sigma_j}$$
> 위에서 sc.fit은 training data의 평균과 표준편차를 구하는 과정이고      
sc.transform을 통해 실제 각 변수마다 표준화된 값이 계산된다.

In [4]:
from sklearn.linear_model import Perceptron
ppn = Perceptron(n_iter=40, eta0=0.1, random_state=0)
ppn.fit(X_train_std, y_train)

Perceptron(alpha=0.0001, class_weight=None, eta0=0.1, fit_intercept=True,
      n_iter=40, n_jobs=1, penalty=None, random_state=0, shuffle=True,
      verbose=0, warm_start=False)

> 이제 데이터를 사용할 준비가 다 되었다.    
Chapter2에는 Perceptrons를 직접 구현 했었는데,   
대신 scikit-learn에 구현되어 있는 것을 사용할 수 있다.   
Chapter2에서 구현한 것과 거의 유사한 형태로 사용하는 것을 확인할 수 있다.

In [5]:
ppn.classes_

array([0, 1, 2])

In [6]:
ppn.coef_

array([[-0.14874662, -0.11229737],
       [ 0.22431308, -0.20230671],
       [ 0.50351128,  0.28717266]])

> 학습한 모형 파라미터는 아래와 같다.    
target이 3가지인 multi-class classification이므로     
One-vs.-Rest의 모형 3가지 사용되어    
weight쌍이 3개 존재함을 알 수 있다.

In [7]:
y_pred = ppn.predict(X_test_std)
print(len(y_pred))
print(y_pred)
print(y_test)
print('Misclassified samples: %d' % (y_test != y_pred).sum())

45
[2 1 0 2 0 2 0 1 1 1 1 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 0 0 2 1 0 2 2 0 0
 2 1 1 2 0 2 0 0]
[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 1 1 1 2 0 2 0 0]
Misclassified samples: 4


> 이렇게 (training data로) 학습시킨 Perceptrons 모형을 이용해     
test data에는 어떤 예측을 하는지 살펴보면, 45건중 41건을 잘 예측함을 확인할 수 있다.

> 오분류율(missclassification error rate)이 $4/45 \approx 0.089$임을 알 수 있다.    
혹은 정확도(accuracy)가 $41/45 \approx 0.911$이라고 할 수 있다.

In [8]:
from sklearn.metrics import accuracy_score
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Accuracy: 0.91


###### Overfitting

앞서 training data를 이용해 모형을 학습시킨 후, test data를 이용해 모형의 예측 정확도를 평가 했었다.

만약 어떤 모형이 training data에는 정말 잘 들어맞는데,     
training data가 아닌 다른 데이터에 대해서는 좋은 예측을 하지 못한다면    
이는 좋은 모형이라고 할 수 없다.    
(비유적으로 100번 풀어본 문제는 항상 잘 맞추는데, 새로운 문제는 잘 못 맞춘다면 이는 좋은 학습이라 할 수 없다.)

이처럼 training data에만 잘 예측 상황을 **overfitting**(과적합)이라 하며 좋은 모형이라 할 수 없다.

반대로 바람직한 상황으로 test data에도 잘 예측하는 모형을 **generalization**(일반화)이 좋은 모형이라 한다.

###### Decision regions

Chapter2에서 했던 것처럼 각 data sample을 잘 분류하는지 살펴보자.

In [9]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

def plot_decision_regions(X, y, classifier, test_idx=None, resolution=0.02):
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution), np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1], alpha=0.8, c=cmap(idx), marker=markers[idx], label=cl)
        

    # highlight test samples
    if test_idx:
        X_test, y_test = X[test_idx, :], y[test_idx]
        plt.scatter(X_test[:, 0], X_test[:, 1], c='',
                   alpha=1.0, linewidth=1, marker='o',
                   s=55, label='test set')
        

In [10]:
X_combined_std = np.vstack((X_train_std, X_test_std))
y_combined = np.hstack((y_train, y_test))

In [11]:
plot_decision_regions(X=X_combined_std,
                     y=y_combined,
                     classifier=ppn,
                     test_idx=range(105,150))

In [ ]:
plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')
plt.show()

### Modeling class probabilities via <u>logistic regression</u>

###### <u>Logistic regression</u> intuition and conditional probabilities

###### Learning the weights of the logistic cost function

.

###### Training a logistic regression model with scikit-learn

###### Tackling overfitting via regularization

.

.

.

.

### Maximum margin classification with <u>support vector machines</u>

###### Maximum margin intuition

###### Dealing with the nonlinearly separable case using slack variables

###### Alternative implementation in scikit-learn

.

.

.

.

.

.

.

### Solving nonlinear problems using <u>kernel SVM</u>

###### Using the kernel trick to find separating hyperplanes in higher dimensional space

.

.

.

.

### <u>Decision tree learning</u>

###### Maximizing information gain - getting the most bang for the buck

###### Building a decision tree

###### Combining weak to strong learners via random forests

.

.

.

.

### K-nearest neighbors - a lazy learning algorithm

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.